In [1]:
from joblib import Parallel, delayed
from math import sqrt
from pathlib import Path
from lxml import etree
from tqdm import tqdm
import pickle
import dateparser

from bs4 import BeautifulSoup

import pandas as pd
import swifter

import pandas as pd
import sqlite3
from cleantext import clean

from pathlib import Path
import numpy as np
import swifter
from somajo import Tokenizer, SentenceSplitter
from tqdm import tqdm
import multiprocessing
from joblib import Parallel, delayed

from german_lemmatizer import lemmatize

num_cores = multiprocessing.cpu_count()

import numpy as np

/home/filter/anaconda3/envs/ptf/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [ ]:
df = pd.read_pickle('/mnt/data2/ptf/cleaned.pkl')

In [ ]:
df = df.drop(columns=['cleaned'])

In [2]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [3]:
def get_sents(texts):
    tokenizer = Tokenizer(split_camel_case=True, token_classes=False, extra_info=False)
    sentence_splitter = SentenceSplitter(is_tuple=False)
    
    results = []
    for text in texts:
        text = clean(text, lang='de', lower=False)
        tokens = tokenizer.tokenize_paragraph(text)
        sentences = sentence_splitter.split(tokens)
        cleaned = [' '.join(s) for s in sentences]
        results.append(cleaned)
    return results

In [4]:
def _fl(a):
    for x in a:
        for xx in x:
            for xxx in xx:
                yield xxx

def flatten_array(a):
    return list(_fl(a))

In [ ]:
sents = Parallel(n_jobs=4, backend='multiprocessing')(delayed(get_sents)(row) for row in tqdm(list(chunks(df['text'].values, 10000))))

In [ ]:
sents = flatten_array(sents)

In [ ]:
pickle.dump( sents, open( "s.pkl", "wb" ) )

In [2]:
sents = pickle.load( open( "s.pkl", "rb" ) )

In [3]:
sents[:10]

['Wer darf Atommacht sein , wer nicht .',
 'Da treffen sich Regierungen , Atommächte und Anwärter auf die Bombe .',
 'Über was wird da verhandelt ?',
 'Die Bombe selbst steht nicht zur Disposition , für die Atommächte , sondern der Verfügungsanspruch eines Anwärters .',
 'Der Besitz dieser Bombe verändert die politischen Optionen eines Staates , und damit auch die militärischen , in der Folge die politischen Optionen der existierenden Atommächte .',
 'Bereits der Wille zur Bombe wird deshalb von den realen Atommächten beaufsichtigt .',
 'Diese Mächte verhalten sich zum Willen eines ausländischen Souveräns wie Polizei .',
 'Wer nicht gehorcht wird bestraft .',
 'Das können diese Mächte , weil diese in der Lage sind ihre Ansprüche an das Wohlverhalten anderer Regierungen wirtschaftlich und militärisch zu erzwingen .',
 'Von wegen hier ginge es um den Schutz vor einer militärischen Bedrohung .']

In [ ]:
slem_txt = list(lemmatize(sents, n_jobs=3, chunk_size=100000))

  7%|▋         | 20/273 [09:41<1:48:33, 25.75s/it]

failed, next try! Command '--line' in image 'filter/german-lemmatizer:0.4.0' returned non-zero exit status 139: b''


 38%|███▊      | 103/273 [1:03:15<1:24:40, 29.89s/it]

In [ ]:
pickle.dump( slem_txt, open( "s_l.pkl", "wb" ) )

In [ ]:
cl = lambda x: clean(x, no_urls=True, no_digits=True, no_punct=True, no_line_breaks=True, lang='de') 

In [ ]:
slem_txt = pickle.load( open( "lemma.pkl", "rb" ) )

In [ ]:
results = pickle.load( open( "results_lemma.pkl", "rb" ) )

In [ ]:
with open('/mnt/data2/ptf/zo2.txt', 'w') as outfile:
    for r in results:
        for d in r:
            outfile.write('\n'.join([ x for x in d]) + '\n')

In [ ]:
pickle.dump( results, open( "results_lemma.pkl", "wb" ) )

In [ ]:
len(results)

In [ ]:
len(slem_txt)

In [ ]:
# fucked up, need to either remove new lines or escape them for 

In [ ]:
for r in results[:100]:
    for d in r:
        print("\n".join(d))

In [5]:
STOP_WORDS = set(
    """
á a ab aber ach acht achte achten achter achtes ag alle allein allem allen
aller allerdings alles allgemeinen als also am an andere anderen anderem andern
anders auch auf aus ausser außer ausserdem außerdem
bald bei beide beiden beim beispiel bekannt bereits besonders besser besten bin
bis bisher bist
da dabei dadurch dafür dagegen daher dahin dahinter damals damit danach daneben
dank dann daran darauf daraus darf darfst darin darüber darum darunter das
dasein daselbst dass daß dasselbe davon davor dazu dazwischen dein deine deinem
deiner dem dementsprechend demgegenüber demgemäss demgemäß demselben demzufolge
den denen denn denselben der deren derjenige derjenigen dermassen dermaßen
derselbe derselben des deshalb desselben dessen deswegen dich die diejenige
diejenigen dies diese dieselbe dieselben diesem diesen dieser dieses dir doch
dort drei drin dritte dritten dritter drittes du durch durchaus dürfen dürft
durfte durften
eben ebenso ehrlich eigen eigene eigenen eigener eigenes ein einander eine
einem einen einer eines einigeeinigen einiger einiges einmal einmaleins elf en
ende endlich entweder er erst erste ersten erster erstes es etwa etwas euch
früher fünf fünfte fünften fünfter fünftes für
gab ganz ganze ganzen ganzer ganzes gar gedurft gegen gegenüber gehabt gehen
geht gekannt gekonnt gemacht gemocht gemusst genug gerade gern gesagt geschweige
gewesen gewollt geworden gibt ging gleich gott gross groß grosse große grossen
großen grosser großer grosses großes gut gute guter gutes
habe haben habt hast hat hatte hätte hatten hätten heisst heißt her heute hier
hin hinter hoch
ich ihm ihn ihnen ihr ihre ihrem ihren ihrer ihres im immer in indem
infolgedessen ins irgend ist
ja jahr jahre jahren je jede jedem jeden jeder jedermann jedermanns jedoch
jemand jemandem jemanden jene jenem jenen jener jenes jetzt
kam kann kannst kaum kein keine keinem keinen keiner kleine kleinen kleiner
kleines kommen kommt können könnt konnte könnte konnten kurz
lang lange leicht leider lieber los
machen macht machte mag magst man manche manchem manchen mancher manches mehr
mein meine meinem meinen meiner meines mensch menschen mich mir mit mittel
mochte möchte mochten mögen möglich mögt morgen muss muß müssen musst müsst
musste mussten
na nach nachdem nahm natürlich neben nein neue neuen neun neunte neunten neunter
neuntes nicht nichts nie niemand niemandem niemanden noch nun nur
ob oben oder offen oft ohne
recht rechte rechten rechter rechtes richtig rund
sagt sagte sah satt schlecht schon sechs sechste sechsten sechster sechstes
sehr sei seid seien sein seine seinem seinen seiner seines seit seitdem selbst
selbst sich sie sieben siebente siebenten siebenter siebentes siebte siebten
siebter siebtes sind so solang solche solchem solchen solcher solches soll
sollen sollte sollten sondern sonst sowie später statt
tag tage tagen tat teil tel trotzdem tun
über überhaupt übrigens uhr um und uns unser unsere unserer unter
vergangene vergangenen viel viele vielem vielen vielleicht vier vierte vierten
vierter viertes vom von vor
wahr während währenddem währenddessen wann war wäre waren wart warum was wegen
weil weit weiter weitere weiteren weiteres welche welchem welchen welcher
welches wem wen wenig wenige weniger weniges wenigstens wenn wer werde werden
werdet wessen wie wieder will willst wir wird wirklich wirst wo wohl wollen
wollt wollte wollten worden wurde würde wurden würden
zehn zehnte zehnten zehnter zehntes zeit zu zuerst zugleich zum zunächst zur
zurück zusammen zwanzig zwar zwei zweite zweiten zweiter zweites zwischen
""".split()
)

In [2]:
def to_txt(data, name):
    text = '\n'.join(data)
    print(len(text))
    Path('/mnt/data2/ptf/co_exp/' + name + '.txt').write_text(text)

In [2]:
sents = pickle.load( open( "s.pkl", "rb" ) )

In [3]:
def cl(x):
    return clean(x, no_urls=True, no_digits=True, no_punct=True, no_line_breaks=True, lang='de') 

In [ ]:
sents = Parallel(n_jobs=4, backend='multiprocessing')(delayed(cl)(row) for row in tqdm(sents))

  5%|▌         | 1418762/27200369 [01:29<28:18, 15174.59it/s]

In [ ]:
pickle.dump( sents, open( "s_cleaned.pkl", "wb" ) )

In [4]:
len(sents)

27200369

In [ ]:
to_txt(sents, 'orig')